In [25]:
%pip install ipywidgets
%pip install sentencetransformers
%pip install pyquery
%pip install tqdm
%pip install nltk
%pip install gensim

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ERROR: Could not find a version that satisfies the requirement sentencetransformers (from versions: none)
ERROR: No matching distribution found for sentencetransformers
Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...

Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.0/24.0 MB 14.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 5.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
from pyquery import PyQuery as pq
import requests
import asyncio
from typing import Any, AsyncGenerator, List
from urllib.parse import urljoin
import json
import unicodedata
from tqdm.notebook import tqdm

TIMES_URL = "https://time.com/"
TIMES_HOME_URL = urljoin(TIMES_URL, "section/us/")
TIMES_PAGE_URL = "/section/us/?page={page}"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:128.0) Gecko/20100101 Firefox/128.0"
}
N_ARTICLES = 20
SUMMARIZED_ARTICLES_PICKLE = "summarized_articles.pkl"
ARTICLES_CORPUS_PICKLE = "articles_with_corpus.pkl"
ARTICLES_PICKLE = "articles.pkl"

In [6]:
def json_str(cls):
    def custom_str(self):
        class_name = self.__class__.__name__
        properties = {k: v for k, v in self.__dict__.items()}
        return json.dumps({"class": class_name, "properties": properties}, indent=4)
    cls.__str__ = custom_str
    cls.__repr__ = custom_str
    return cls

def remove_unicode_and_normalize(s: str) -> str:
    normalized = unicodedata.normalize('NFKD', s)
    return normalized.encode('ascii', 'ignore').decode('ascii')

In [7]:
@json_str
class RateLimiter:
    def __init__(self, n: int=1, interval:float=2):
        self.n = n
        self.interval = interval
        self.calls = 0
        self.lock = asyncio.Lock()

    async def acquire(self) -> None:
        async with self.lock:
            if self.calls >= self.n:
                await asyncio.sleep(self.interval)
                self.calls = 0
            self.calls += 1

In [8]:
from typing import TypeVar


T = TypeVar('T')
async def async_islice(generator: AsyncGenerator[T, None], stop: int, start: int = 0) -> AsyncGenerator[T, None]:
    i = -1
    async for value in generator:
        i += 1
        if i < start:
            continue
        if i >= stop:
            break
        yield value

In [9]:
@json_str
class Article:
    def __init__(self, url: str, title: str, paragraphs: List[str]):
        self.paragraphs: List[str] = paragraphs
        self.title: str = title
        self.url: str = url

In [10]:
async def iter_times(max_pages: int = 10) -> AsyncGenerator[Article, None]:
    rl = RateLimiter(interval=1)

    def get_page_from_url(url) -> pq:
        resp = requests.get(url, headers=HEADERS)
        if resp.status_code != 200:
            return None
        page = pq(resp.text)
        return page
    
    async def iter_pages(start_url: str) -> AsyncGenerator[pq, None]:
        page_num = 1
        
        def get_next_page() -> pq:
            nonlocal page_num
            page_num += 1
            page = get_page_from_url(urljoin(TIMES_URL, TIMES_PAGE_URL.format(page=page_num)))
            return page
        
        await rl.acquire()
        page = get_page_from_url(start_url)
        await rl.acquire()
        next_page = get_next_page()
        yield page

        while next_page is not None:
            yield page
            await rl.acquire()
            page = next_page
            next_page = get_next_page()

    def get_article_urls(head_page: str) -> List[str]:
        article_urls = [x.attr("href") for x in head_page("div.component div.taxonomy-tout a").items()]
        return article_urls
    
    def get_article_from_url(url: str) -> Article:
        page = get_page_from_url(url)
        try:
            title = remove_unicode_and_normalize(page("h1.self-baseline").text())
            paragraphs = [remove_unicode_and_normalize(x.text()) for x in page("p.self-baseline.px-0").items()]
            return Article(url, title, paragraphs)
        except Exception as e:
            print(f"Article at url='{url}' failed to parse: {e}")
            return None

    async for page in async_islice(iter_pages(TIMES_HOME_URL), max_pages):
        urls = get_article_urls(page)
        for url in urls:
            yield get_article_from_url(urljoin(TIMES_URL, url))
            await rl.acquire()
    

In [45]:
N_ARTICLES = 100

articles: List[Article] = []
with tqdm(total=N_ARTICLES) as pbar:
    async for article in async_islice(iter_times(), N_ARTICLES):
        pbar.update(1)
        if article is not None:
            articles.append(article)
    tqdm.write(f"Processed {len(articles)}/{N_ARTICLES} before iterator ended.")


import pickle
with open(ARTICLES_PICKLE, "wb") as f:
    pickle.dump(articles, f)

  0%|          | 0/100 [00:00<?, ?it/s]

Processed 100/100 before iterator ended.


In [49]:
import pickle
with open(ARTICLES_PICKLE, "rb") as f:
    articles: List[Article] = pickle.load(f)

In [11]:
@json_str
class SummarizedArticle(Article):    
    def __init__(self, summary: str, **kwargs):
        super().__init__(**kwargs)
        self.summary: str = summary

In [107]:
from sentence_transformers import SentenceTransformer
from nltk.tokenize import sent_tokenize
import numpy as np
# https://github.com/UKPLab/sentence-transformers/tree/master/examples/applications/text-summarization
from LexRank import degree_centrality_scores

SUMMARY_LENGTH = 3

model = SentenceTransformer("all-mpnet-base-v2")

summarized_articles: List[SummarizedArticle] = []

with tqdm(total=len(articles)) as pbar:
    for article in articles:
        sentences: List[str] = []
        for paragraph in article.paragraphs:
            sentences.extend(sent_tokenize(paragraph))

        sentences = np.array(sentences)
        
        embeddings = model.encode(sentences)

        # https://github.com/UKPLab/sentence-transformers/blob/master/examples/applications/text-summarization/text-summarization.py
        similarity_scores = model.similarity(embeddings, embeddings).numpy()
        centrality_scores = degree_centrality_scores(similarity_scores, threshold=None)
        most_central_sentence_indices = np.argsort(-centrality_scores)

        summary = "\n".join(sentences[most_central_sentence_indices[0:SUMMARY_LENGTH]])
        summarized_articles.append(SummarizedArticle(summary, **article.__dict__))
        pbar.update(1)
    pbar.write(f"Summarized {len(summarized_articles)} articles.")

import pickle

with open(SUMMARIZED_ARTICLES_PICKLE, "wb") as f:
    pickle.dump(summarized_articles, f)

  0%|          | 0/100 [00:00<?, ?it/s]

Summarized 100 articles.


In [108]:
import pickle
with open(SUMMARIZED_ARTICLES_PICKLE, "rb") as f:
    summarized_articles: List[SummarizedArticle] = pickle.load(f)

In [109]:
print("\n\n".join([s.summary for s in summarized_articles[0:2]]))

Besides breaching the Trump campaign, officials also believe that Iran tried to hack into the presidential campaign of Kamala Harris.
We have observed increasingly aggressive Iranian activity during this election cycle, specifically involving influence operations targeting the American public and cyber operations targeting Presidential campaigns, said the statement released by the FBI, the Office of the Director of National Intelligence and the Cybersecurity and Infrastructure Security Agency.
Earlier this month, Microsoft issued a report detailing foreign agents attempts to interfere in this years election, citing an instance of an Iranian military intelligence unit in June sending a spear-phishing email to a high-ranking official of a presidential campaign from a compromised email account of a former senior advisor.

As the Democratic Party finds itself once again on the cusp of potentially electing its first female President, Clinton, now 76, used her speech to rally behind Vice Pre

In [110]:
import nltk
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
from nltk.probability import FreqDist


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

#https://stackoverflow.com/questions/43742956/fast-named-entity-removal-with-nltk
def fast_ne_removal(tokens):
    tagged = pos_tag(tokens)
    tree = ne_chunk(tagged)
    non_entities = [leaf[0] for leaf in tree if type(leaf) != nltk.Tree]
    return non_entities

def clean_text(text):
    text = re.sub(r'\d+', '', text)
    words = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word.isalnum() and word not in stop_words]
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    freq_dist = FreqDist(words)
    words = [word for word in words if freq_dist[word] > 1]
    
    words = fast_ne_removal(words)
    words = [s.lower() for s in words]
    return words

[nltk_data] Downloading package punkt to /Users/garret/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/garret/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/garret/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/garret/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/garret/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /Users/garret/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [12]:
@json_str
class ArticleWithCorpus(SummarizedArticle):
    def __init__(self, corpus: List[str], **kwargs):
        super().__init__(**kwargs)
        self.corpus : List[str] = corpus

In [111]:
articles_corpus: List[ArticleWithCorpus] = [ArticleWithCorpus(clean_text("\n".join(s.paragraphs)), **s.__dict__) for s in summarized_articles]

import pickle
with open(ARTICLES_CORPUS_PICKLE, "wb") as f:
    pickle.dump(articles_corpus, f)

In [13]:
import pickle
with open(ARTICLES_CORPUS_PICKLE, "rb") as f:
    articles_corpus: List[ArticleWithCorpus] = pickle.load(f)

In [14]:
for a in articles_corpus:
    print(a.corpus)
    print("\n")

['intelligence', 'official', 'said', 'confident', 'responsible', 'hack', 'presidential', 'campaign', 'cyber', 'effort', 'interfere', 'outcome', 'election', 'the', 'federal', 'time', 'hack', 'foreign', 'election', 'the', 'hacking', 'also', 'top', 'official', 'democratic', 'process', 'campaign', 'official', 'also', 'tried', 'hack', 'presidential', 'campaign', 'read', 'more', 'the', 'gaza', 'the', 'hacking', 'activity', 'federal', 'official', 'said', 'foreign', 'interest', 'the', 'democratic', 'influence', 'outcome', 'election', 'interest', 'official', 'said', 'activity', 'election', 'cycle', 'influence', 'operation', 'targeting', 'cyber', 'operation', 'targeting', 'presidential', 'campaign', 'said', 'statement', 'released', 'hack', 'interfere', 'election', 'evidence', 'the', 'statement', 'released', 'time', 'threatened', 'threatened', 'strike', 'official', 'strike', 'last', 'month', 'top', 'strike', 'more', 'gaza', 'the', 'detail', 'responsible', 'trump', 'campaign', 'said', 'intelligenc

In [114]:
from sentence_transformers import util

model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode([" ".join(a.corpus) for a in articles_corpus], batch_size=64, show_progress_bar=True, convert_to_tensor=True)
clusters = util.community_detection(embeddings, min_community_size=3, threshold=0.5)

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

In [115]:
for i, cluster in enumerate(clusters):
    print(f"({i + 1}) {len(cluster)} Articles")
    for sentence_id in cluster[0:3]:
        print("\t", articles_corpus[sentence_id].title)
    print("\t...")

(1) 14 Articles
	 
	 Who Could Be Kamala Harris Running Mate? Heres the Shortlist The Shortlist Other Contenders Out of Consideration
	 J.D. Vance Tells RNC Hell Be a Vice President Who Never Forgets Where He Came From
	...
(2) 5 Articles
	 
	 The Prosecutor Versus Felon Narrative Helps No One
	 J.D. Vances Selection as Trumps Running Mate Frightens Business Leaders
	...
(3) 4 Articles
	 Tropical Storm Debby Moving Through Gulf Toward Florida With Hurricane Warnings
	 Remnants of Debby Threaten the Northeast With Flooding and Possible Tornadoes
	 Puerto Rico Cancels Classes, Activates National Guard as Tropical Storm Ernesto Approaches
	...
(4) 4 Articles
	 Senate Set to Pass Bipartisan Bill Meant to Protect Childrens Online Safety
	 The Department of Justice Sued TikTok
	 Madhvi Chittoor
	...
(5) 4 Articles
	 Records Show Deputy Charged in Sonya Masseys Fatal Shooting Worked for Six Agencies in 4 years
	 Autopsy Confirms Sonya Massey Died From a Gunshot Wound to the Head
	 Authorities

In [ ]:
%pip install ollama

In [17]:
import ollama

response = ollama.chat(model='phi3', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response['message']['content'])

The sky appears blue to us because of a process called Rayleigh scattering. As sunlight, which contains all colors of light, enters Earth's atmosphere, it collides with molecules and tiny particles in the air. These particles are much smaller than the wavelength of visible light, so they scatter shorter-wavelength colors (blue and violet) more efficiently than longer-wavelength ones (like red and yellow).

However, our eyes are not as sensitive to violet light as they are to blue light. Moreover, some ozone absorbs the violet light, which also contributes to making the sky look blue instead of violet. So while all colors contribute to the color we see, it is predominantly blue due to a combination of atmospheric effects and human perception.

As sunlight passes through Earth's atmosphere and reaches our eyes from different angles throughout the day or during various weather conditions (like when rainbow appears), there can be slight variations in sky colors, but generally, it is percei